# LangChain 核心模块 Agent - Self ask with search

- Google 搜索对接
    - 第三方平台：https://serpapi.com
    - LangChain API 封装：`SerpAPIWrapper`
- LangChain Agent 模块使用
    - Agent 类型：`SELF_ASK_WITH_SEARCH`
    - Agent 实例化：`initialize_agent` 方法
 
https://python.langchain.com/docs/modules/agents/agent_types/

In [1]:
import os

# 更换为自己的 Serp API KEY
os.environ["SERPAPI_API_KEY"] = "c888b7ef0d3d37738c7a98297aab0563acd8de0253460609a89726b15aca0055"

In [2]:
from langchain_openai import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool

llm = OpenAI(temperature=0)

In [3]:
# 实例化查询工具
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

In [6]:
# 实例化 SELF_ASK_WITH_SEARCH Agent
self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True, handle_parsing_errors=True
)

In [7]:
# 实际运行 Agent，查询问题（正确）
self_ask_with_search.run(
    "成都举办的大运会是第几届大运会？"
)



> Entering new AgentExecutor chain...
Could not parse output:  No. 
Final answer: 第28届大运会 (28th Universiade Games)
Intermediate answer: Invalid or incomplete response
Could not parse output: 
Intermediate answer: Invalid or incomplete response
So the final answer is: 第28届大运会

> Finished chain.


'第28届大运会'

In [8]:
# 实际运行 Agent，查询问题（有歧义）
self_ask_with_search.run(
    "2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
Could not parse output:  No.
Intermediate answer: Invalid or incomplete response
So the final answer is: Invalid or incomplete response

> Finished chain.


'Invalid or incomplete response'

In [9]:
# Reason-only 答案（错误）
print(llm("成都举办的大运会是第几届大运会？"))

c:\ProgramData\Anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




成都举办的大运会是第31届大运会。


In [10]:
# Reason-only 答案（错误）
print(llm("2023年大运会举办地在哪里？"))

 

2023年大运会的举办地尚未确定。目前，有五个城市在争夺2023年大运会的主办权，分别是阿根廷的科尔多瓦、哥伦比亚的卡利、南非的比勒陀利亚、泰国的曼谷和委内瑞拉的莫雷利亚。最终的决定将由国际大学生体育联合会（FISU）做出。


#### 使用 GPT-4 作为大语言模型实现更优的 ReAct 范式

In [11]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

In [12]:
self_ask_with_search_chat = initialize_agent(
    tools, chat_model, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [13]:
# GPT-4 based ReAct 答案（正确）
self_ask_with_search_chat.run(
    "2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
Yes.
Follow up: What is the English translation of "2023年大运会举办地"?
Intermediate answer: The translation is "The location of the 2023 Universiade."
Follow up: Where is the 2023 Universiade being held?
Intermediate answer: The 2023 Universiade is being held in Chengdu, China.
So the final answer is: Chengdu, China

> Finished chain.


'Chengdu, China'